In [1]:
# it works only on Unix

import signal
import time
from contextlib import contextmanager


class TimeoutException(Exception): pass


@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")

    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)


def long_function_call():
    i = 0
    while True:
        print(f"Step done {i}")
        i += 1
        time.sleep(1)


try:
    with time_limit(10):
        long_function_call()
except TimeoutException as e:
    print("Timed out!")

AttributeError: module 'signal' has no attribute 'SIGALRM'

In [1]:
from contextlib import contextmanager
import threading
import _thread


class TimeoutException(Exception):
    def __init__(self, msg=''):
        self.msg = msg


@contextmanager
def time_limit(seconds, msg=''):
    timer = threading.Timer(seconds, lambda: _thread.interrupt_main())
    timer.start()
    try:
        yield
    except KeyboardInterrupt:
        raise TimeoutException("Timed out for operation {}".format(msg))
    finally:
        # if the action ends in specified time, timer is canceled
        timer.cancel()


import time

# ends after 5 seconds
for stage in range(3):
    print(stage)
    try:
        with time_limit(5, 'sleep'):
            for i in range(10):
                time.sleep(1)
                print(f"step {i}")
    except TimeoutException:
        pass

# this will actually end after 10 seconds
with time_limit(5, 'sleep'):
    print("Doing")
    time.sleep(10)
    print("DONE")

0
step 0
step 1
step 2
step 3
step 4
1
step 0
step 1
step 2
step 3
step 4
2
step 0
step 1
step 2
step 3
step 4
Doing


TimeoutException: Timed out for operation sleep